### Import custom modules from current folder

In [1]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))

if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import nltk
from sklearn.model_selection import train_test_split
from simple_text_representation.classes import Text
from simple_text_representation.models import Database
import numpy as np
# from nltk.draw.tree import draw_trees

In [3]:
database = Database('educationalTexts', 'postgres', '', '0.0.0.0', 5432)
path = r'http://localhost/'

In [4]:
def transformToString(text):
    textStr = ''

    for paragraph in text:
        for line in paragraph:
            textStr = textStr + line
    return textStr

In [5]:
textOfSeventhGrade = Text.getTexts(database, grade=7)
textOfEightGrade = Text.getTexts(database, grade=8)
textOfNineGrade = Text.getTexts(database, grade=9)

textsFormatedSG = [transformToString(textArr) for textArr in textOfSeventhGrade]
textsFormatedEG = [transformToString(textArr) for textArr in textOfEightGrade]
textsFormatedNG = [transformToString(textArr) for textArr in textOfNineGrade]

In [6]:
textOfSeventhGrade[0]

[['Por lo tanto, las campañas públicas deben esclarecer la naturaleza de los procedimientos técnicos y legales, para disipar miedos; pero, esencialmente, deben apuntar a que se tome conciencia de lo que significa salvar otra vida, porque para decidirlo en un momento crucial es necesario que la idea se haya considerado y discutido previamente, con reflexión y calma.'],
 ['La última cuestión es la más compleja; en la actualidad, aunque alguien haya manifestado expresamente su voluntad de donar, es a la familia a la que se consulta en el momento en que la donación puede efectuarse.',
  ' Como se entiende, tal consulta llega en un momento difícil y poco propicio para las reflexiones profundas, más aún si se tiene que tomar una decisión rápida.'],
 ['Es preciso, entonces, que se aclaren algunas dudas para que las personas pierdan el miedo a donar.',
  ' Primero, que lo complicado de los procedimientos de extirpación y trasplantación, en el que intervienen varios equipos médicos altamente es

In [7]:
textsFormatedSG[0]

'Por lo tanto, las campañas públicas deben esclarecer la naturaleza de los procedimientos técnicos y legales, para disipar miedos; pero, esencialmente, deben apuntar a que se tome conciencia de lo que significa salvar otra vida, porque para decidirlo en un momento crucial es necesario que la idea se haya considerado y discutido previamente, con reflexión y calma.La última cuestión es la más compleja; en la actualidad, aunque alguien haya manifestado expresamente su voluntad de donar, es a la familia a la que se consulta en el momento en que la donación puede efectuarse. Como se entiende, tal consulta llega en un momento difícil y poco propicio para las reflexiones profundas, más aún si se tiene que tomar una decisión rápida.Es preciso, entonces, que se aclaren algunas dudas para que las personas pierdan el miedo a donar. Primero, que lo complicado de los procedimientos de extirpación y trasplantación, en el que intervienen varios equipos médicos altamente especializados, vuelve muy dif

### Format train and test data

In [8]:
data = np.concatenate((np.array(textsFormatedSG), np.array(textsFormatedEG), np.array(textsFormatedNG))) 
labels = np.concatenate((np.full(len(textsFormatedSG), 0), np.full(len(textsFormatedEG), 1), np.full(len(textsFormatedNG), 2)))

In [44]:
data[0]

'Por lo tanto, las campañas públicas deben esclarecer la naturaleza de los procedimientos técnicos y legales, para disipar miedos; pero, esencialmente, deben apuntar a que se tome conciencia de lo que significa salvar otra vida, porque para decidirlo en un momento crucial es necesario que la idea se haya considerado y discutido previamente, con reflexión y calma.La última cuestión es la más compleja; en la actualidad, aunque alguien haya manifestado expresamente su voluntad de donar, es a la familia a la que se consulta en el momento en que la donación puede efectuarse. Como se entiende, tal consulta llega en un momento difícil y poco propicio para las reflexiones profundas, más aún si se tiene que tomar una decisión rápida.Es preciso, entonces, que se aclaren algunas dudas para que las personas pierdan el miedo a donar. Primero, que lo complicado de los procedimientos de extirpación y trasplantación, en el que intervienen varios equipos médicos altamente especializados, vuelve muy dif

In [9]:
labels

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2])

### Preprocesing the data

In [10]:
import keras
import keras.preprocessing.text as kpt
from keras.preprocessing.text import Tokenizer

/Users/herbert/.pyenv/versions/anaconda3-5.0.1/envs/jupyter3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [45]:
maxWords = 3000
tokenizer = Tokenizer(num_words=maxWords)
tokenizer.fit_on_texts(data)
dictionary = tokenizer.word_index

In [46]:
def convert_text_to_index_array(text):
    return [dictionary[word] for word in kpt.text_to_word_sequence(text)]

In [47]:
allWordIndices = []

for text in data:
    wordIndices = convert_text_to_index_array(text)
    allWordIndices.append(wordIndices)

In [48]:
allWordIndices = np.asarray(allWordIndices)

In [49]:
new_data = tokenizer.sequences_to_matrix(allWordIndices)
new_labels = keras.utils.to_categorical(labels, 3)

### Creating the model

In [50]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation

In [51]:
model = Sequential()
model.add(Dense(512, input_shape=(maxWords,), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='sigmoid'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))

In [52]:
model.compile(loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy'])

In [53]:
model.fit(new_data, new_labels,
  batch_size=32,
  epochs=5,
  verbose=1,
  validation_split=0.1,
  shuffle=True)

Train on 120 samples, validate on 14 samples
Epoch 1/5
120/120 [==============================] - 1s 4ms/step - loss: 1.3913 - acc: 0.4333 - val_loss: 1.9243 - val_acc: 0.0000e+00
Epoch 2/5
120/120 [==============================] - 0s 364us/step - loss: 1.1022 - acc: 0.4583 - val_loss: 1.9742 - val_acc: 0.0000e+00
Epoch 3/5
120/120 [==============================] - 0s 449us/step - loss: 0.9101 - acc: 0.5667 - val_loss: 1.8956 - val_acc: 0.0000e+00
Epoch 4/5
120/120 [==============================] - 0s 454us/step - loss: 0.7647 - acc: 0.6417 - val_loss: 1.9417 - val_acc: 0.0000e+00
Epoch 5/5
120/120 [==============================] - 0s 667us/step - loss: 0.5870 - acc: 0.7750 - val_loss: 1.8392 - val_acc: 0.0000e+00


In [30]:
textOfTenthGrade = Text.getTexts(database,grade=10)

In [33]:
txtStr = transformToString(textOfTenthGrade[0])
txtStr

'Son casos especiales en que puede hacerse uso del temor aquellos en que el anuncio tenga como uno de sus objetivos promover la prudencia en determinadas actividades o desalentar conductas o actividades consideradas contrarias a la seguridad o salud del consumidor.Artículo 14°.- Explotación de la Superstición y Temor La publicidad deberá evitar toda forma de explotación de la superstición, la ignorancia, la credulidad y, excepto en casos especiales, del temor de los consumidores.Artículo 13°.- Publicidad de servicios de contenido erótico La publicidad que promociona servicios de contenido erótico no debe ser difundida a un público distinto al adulto. Por tanto, la difusión de este tipo de publicidad está restringida a lugares, medios, soportes, espacios u horarios de circulación restringida para adultos.La publicidad no deberá inducir a los destinatarios del mensaje publicitario a cometer un acto ilegal o perjudicial para su seguridad personal o la de terceros. El hecho que un producto

In [36]:
def convert_test_text_to_index_array(text):
    words = kpt.text_to_word_sequence(text)
    wordIndices = []
    for word in words:
        if word in dictionary:
            wordIndices.append(dictionary[word])
        else:
            print("'%s' not in training corpus; ignoring." %(word))
    return wordIndices

In [37]:
testArr = convert_test_text_to_index_array(txtStr)
inputTxt = tokenizer.sequences_to_matrix([testArr], mode='binary')

'prudencia' not in training corpus; ignoring.
'desalentar' not in training corpus; ignoring.
'conductas' not in training corpus; ignoring.
'consideradas' not in training corpus; ignoring.
'contrarias' not in training corpus; ignoring.
'14°' not in training corpus; ignoring.
'deberá' not in training corpus; ignoring.
'13°' not in training corpus; ignoring.
'promociona' not in training corpus; ignoring.
'difundida' not in training corpus; ignoring.
'distinto' not in training corpus; ignoring.
'difusión' not in training corpus; ignoring.
'restringida' not in training corpus; ignoring.
'soportes' not in training corpus; ignoring.
'restringida' not in training corpus; ignoring.
'deberá' not in training corpus; ignoring.
'inducir' not in training corpus; ignoring.
'destinatarios' not in training corpus; ignoring.
'terceros' not in training corpus; ignoring.
'indecente' not in training corpus; ignoring.
'objetar' not in training corpus; ignoring.
'validez' not in training corpus; ignoring.
'd

In [41]:
pred = model.predict(inputTxt)
print("%s grade; %f%% confidence" % (labels[np.argmax(pred)]+7, pred[0][np.argmax(pred)] * 100))

[[0.33698675 0.4989199  0.16409333]]
7 grade; 49.891990% confidence
